## ipc和clc实验数据自动提取关键词

In [1]:
#读入ipc文件后value中没有空格了（有重复的数据代表不是一一映射，并不是错误）
ipc_dict = {}
ipc_key_list = []
ipc_value_list = []
with open("./ipc/ipc_keywords_sentences.txt", 'r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        line = line.split(' ')
        
        assert len(line) == 2, '错误出现在key：' + str(line[0]) + '!'
        if str(line[0]) in ipc_key_list:
            print('重复的ipc分类号：', str(line[0]))
            
        ipc_key_list.append(str(line[0]))
        ipc_str = ''
        for index in range(1, len(line)):
            ipc_str = ipc_str + str(line[index])
        ipc_dict[str(line[0])] = ipc_str
        ipc_value_list.append(ipc_str)

print('ipc_dict: ', len(ipc_dict))
print('ipc_key_list: ', len(ipc_key_list))
print('ipc_value_list: ', len(ipc_value_list))
print('dup ipc_key_list: ', len(set(ipc_key_list)))
print('dup ipc_value_list: ', len(set(ipc_value_list)))

重复的ipc分类号： A41B17/00
重复的ipc分类号： A01C3/04
重复的ipc分类号： A61K9/20
重复的ipc分类号： A01K77/00
重复的ipc分类号： A01B3/04
重复的ipc分类号： B60L7/22
重复的ipc分类号： B28C5/42
重复的ipc分类号： B64D29/00
重复的ipc分类号： C22C30/02
重复的ipc分类号： E21F13/04
重复的ipc分类号： E01F7/00
重复的ipc分类号： F21W106/00
重复的ipc分类号： F21W106/00
重复的ipc分类号： F02K3/00
重复的ipc分类号： F42C19/08
重复的ipc分类号： G06C15/02
重复的ipc分类号： H02K25/00
ipc_dict:  356
ipc_key_list:  373
ipc_value_list:  373
dup ipc_key_list:  356
dup ipc_value_list:  360


### clc有重复的需要在后面一个clc号上加上  '+1' , ipc重复的ipc分类号不用管

In [2]:
#读入clc文件后value中没有空格了（有重复的数据代表不是一一映射，并不是错误）
clc_dict = {}
clc_key_list = []
clc_value_list = []

with open("./clc/clc_keywords_sentences_dup.txt", 'r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        line = line.split(' ')
        
        assert len(line) == 2, '错误出现在key：' + str(line[0]) + '!'
        if str(line[0]) in clc_key_list:
            print('重复的clc分类号：', str(line[0]))
            
        clc_key_list.append(str(line[0]))
        clc_str = ''
        for index in range(1, len(line)):
            clc_str = clc_str + str(line[index])
        clc_dict[str(line[0])] = clc_str
        clc_value_list.append(clc_str)

print('clc_dict: ', len(clc_dict))
print('clc_key_list: ', len(clc_key_list))
print('clc_value_list: ', len(clc_value_list))
print('dup clc_key_list: ', len(set(clc_key_list)))
print('dup clc_value_list: ', len(set(clc_value_list)))

clc_dict:  373
clc_key_list:  373
clc_value_list:  373
dup clc_key_list:  373
dup clc_value_list:  359


In [3]:
#统计哪些clc的key重复了
from collections import Counter

clc_key_list_dict = dict(Counter(clc_key_list))

print({key:value for key,value in clc_key_list_dict.items() if value > 1})

{}


In [4]:
'''
#对重复clc的key的进行修改，使其不重复
clc_key_dup_dict = {}
for clc_key1 in clc_key_list:
    clc_key_dup_dict[clc_key1] = 0

f_clc_write = open('./clc/clc_keywords_dup.txt', 'w', encoding='utf-8')

for clc_key2, clc_value2 in zip(clc_key_list, clc_value_list):
    clc_key_dup_dict[clc_key2] += 1
    if clc_key_dup_dict[clc_key2] == 1:
        f_clc_write.write(clc_key2 + ' ' + clc_value2 + '\n')
    else:
        f_clc_write.write(clc_key2 + '+' + str(clc_key_dup_dict[clc_key2]-1) + ' ' + clc_value2 + '\n')

f_clc_write.close()
'''

"\n#对重复clc的key的进行修改，使其不重复\nclc_key_dup_dict = {}\nfor clc_key1 in clc_key_list:\n    clc_key_dup_dict[clc_key1] = 0\n\nf_clc_write = open('./clc/clc_keywords_dup.txt', 'w', encoding='utf-8')\n\nfor clc_key2, clc_value2 in zip(clc_key_list, clc_value_list):\n    clc_key_dup_dict[clc_key2] += 1\n    if clc_key_dup_dict[clc_key2] == 1:\n        f_clc_write.write(clc_key2 + ' ' + clc_value2 + '\n')\n    else:\n        f_clc_write.write(clc_key2 + '+' + str(clc_key_dup_dict[clc_key2]-1) + ' ' + clc_value2 + '\n')\n\nf_clc_write.close()\n"

In [5]:
#jieba用来分词和词性标注以及提取关键词
import jieba
import jieba.analyse
import jieba.posseg as pseg

#给结巴添加自定义词典
jieba.load_userdict("./word_embedding/vocab.txt")

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.597 seconds.
Prefix dict has been built successfully.


In [6]:
#要过滤的词性
filter_characters = ['t', 'TIME', 'w', 'm', 'xc', 'nz', 'c', 'p']
#要过滤的中英文符号
ignored_characters = ['()', '{}', '{', '}', '[]', '*', '=', '+', '<>', '<', '>', '&', '#', \
                    '@', '~', '%', '$', '|', '（）', '（', '）', '『』', '『', '』', '【】', \
                    '【', '】', '‘', '￥', '～', '……', '——', '“”', '“', '”', '：', '《', '》', \
                    '《》', '？', '、']
#提取关键字时允许的词性
#allow_pos_characters = ('n', 'f', 's', 'nr', 'ns', 'nt', 'nw', 'v', 'vd', 'vn', \
#                        'a', 'ad', 'an', 'd', 'r', 'p', 'u', 'PER', 'LOC', 'ORG')
allow_pos_characters = ('n', 'f', 's', 'nr', 'ns', 'nt', 'nw', 'v', 'vd', 'vn', \
                        'a', 'ad', 'an', 'd', 'r', 'p', 'u', 'PER', 'LOC', 'ORG',
                       'nz', 'l', 'b', 'nrt', 'z', 'g', 'vg', 'nrfg', 'nrf', 'nis', \
                        'j', 'nhd', 'ng', 'i', 'nnd', 'nit', 'nnt', 'gi', 'gm', 'gp', \
                        'gm', 'gg', 'gb', 'zg', 'nba', 'nnt', 'nf', 'gc', 'nmc', 'mq')

def is_in_allow_pos_characters(flag):
    return True if flag in allow_pos_characters else False

def not_in_filter_characters(flag):
    return True if flag not in filter_characters else False

def not_in_ignored_characters(word):
    return True if word not in ignored_characters else False

### 对clc提取关键词

In [7]:
save_drop_characters_dict = dict()
clc_key_words_list = []


for key, value in zip(clc_key_list, clc_value_list):
    #当value中只有一个字的时候，则不进行提取关键词
    if len(value) > 1:
        #使用TF-IDF算法和TextRank算法提取关键词，并取两者的并集，可以指定提取关键词的个数(topK)和提取关键词的词性(allowPOS)
        list1 = jieba.analyse.extract_tags(value, topK=30, withWeight=False, \
                                        allowPOS=allow_pos_characters)
        list2 = jieba.analyse.textrank(value, topK=30, withWeight=False, \
                                    allowPOS=allow_pos_characters)

        #取并集并去重
        dedup_clc_key_words_list = list(set(list1).union(set(list2)))
        #提取关键词要大于等于１
        if len(dedup_clc_key_words_list) >= 1:
            clc_key_words_str = ''
            for value0 in dedup_clc_key_words_list:
                clc_key_words_str = clc_key_words_str + str(value0) + ';'

            clc_key_words_list.append(str(clc_key_words_str))
        
        #提取不到关键词,则进行分词，由词性标注提取关键词
        else:
            words = pseg.cut(value) #jieba默认模式
            save_drop_characters_dict[key] = ''
            clc_key_words_str = ''
            for word, flag in words:
                if is_in_allow_pos_characters(flag):
                    clc_key_words_str = clc_key_words_str + str(word) + ';'
                else:
                    save_drop_characters_dict[key] = save_drop_characters_dict[key] + str(word) + str(flag) + ';'
            #分词后也要大于1，否则不分词
            if len(clc_key_words_str) >= 1:
                clc_key_words_list.append(str(clc_key_words_str))
            else:
                clc_key_words_list.append(str(value))
                            
    #不提取关键词
    elif len(value) == 1:
        clc_key_words_list.append(str(value))
    else:
        print('error: ', key, value)

In [8]:
len(save_drop_characters_dict)

0

In [9]:
len(clc_key_words_list)

373

In [10]:
f_w = open('./clc/clc_keywords_sentences_dup_by_jieba.txt', 'w', encoding='utf-8')
assert len(clc_key_list) == len(clc_key_words_list), '提取关键词有误!'
for key, value in zip(clc_key_list, clc_key_words_list):
    f_w.write(key + ' ' + value + '\n')
f_w.close()

### 对ipc提取关键词

In [11]:
save_drop_characters_dict = dict()
ipc_key_words_list = []


for key, value in zip(ipc_key_list, ipc_value_list):
    #当value中只有一个字的时候，则不进行提取关键词
    if len(value) > 1:
        #使用TF-IDF算法和TextRank算法提取关键词，并取两者的并集，可以指定提取关键词的个数(topK)和提取关键词的词性(allowPOS)
        list1 = jieba.analyse.extract_tags(value, topK=30, withWeight=False, \
                                        allowPOS=allow_pos_characters)
        list2 = jieba.analyse.textrank(value, topK=30, withWeight=False, \
                                    allowPOS=allow_pos_characters)

        #取并集并去重
        dedup_ipc_key_words_list = list(set(list1).union(set(list2)))
        #提取关键词要大于等于１
        if len(dedup_ipc_key_words_list) >= 1:
            ipc_key_words_str = ''
            for value0 in dedup_ipc_key_words_list:
                ipc_key_words_str = ipc_key_words_str + str(value0) + ';'

            ipc_key_words_list.append(str(ipc_key_words_str))
        
        #提取不到关键词,则进行分词，由词性标注提取关键词
        else:
            words = pseg.cut(value) #jieba默认模式
            save_drop_characters_dict[key] = ''
            ipc_key_words_str = ''
            for word, flag in words:
                if is_in_allow_pos_characters(flag):
                    ipc_key_words_str = ipc_key_words_str + str(word) + ';'
                else:
                    save_drop_characters_dict[key] = save_drop_characters_dict[key] + str(word) + str(flag) + ';'
            #分词后也要大于1，否则不分词
            if len(ipc_key_words_str) >= 1:
                ipc_key_words_list.append(str(ipc_key_words_str))
            else:
                ipc_key_words_list.append(str(value))
                            
    #不提取关键词
    elif len(value) == 1:
        ipc_key_words_list.append(str(value))
    else:
        print('error: ', key, value)

In [12]:
len(save_drop_characters_dict)

0

In [13]:
len(ipc_key_words_list)

373

In [14]:
f_w = open('./ipc/ipc_keywords_sentences_by_jieba.txt', 'w', encoding='utf-8')
assert len(ipc_key_list) == len(ipc_key_words_list), '提取关键词有误!'
for key, value in zip(ipc_key_list, ipc_key_words_list):
    f_w.write(key + ' ' + value + '\n')
f_w.close()

## 自动构造输入文件input.xlsx

In [16]:
#构造ipc输入文件
import pandas as pd
import xlwt
import re

pattern = '[A-H]{1}[0-9]{2}[A-Z]{1}[0-9]{1,4}\/[0-9]{2,6}' #匹配ipc分类号的正则表达式
f_ipc_r = open('./ipc/ipc_keywords.txt', 'r', encoding='utf-8') #关键词
f_clc_r = open('./clc/clc_keywords.txt', 'r', encoding='utf-8') #关键词

clc_list = []
for line in f_clc_r:
    line = line.strip()
    line_list = line.split(' ')
    clc_list.append(str(line_list[0]))
    

writebook = xlwt.Workbook()  #打开一个excel
sheet = writebook.add_sheet('ipc_input')  #在打开的excel中添加一个sheet

sheet.write(0, 0, 'IPC分类号') #写入excel，1行0列
sheet.write(0, 1, 'IPC描述') #写入excel，1行1列
sheet.write(0, 2, 'CLC分类号') #写入excel，1行2列

#控制sheet的行数
i = 0
for line in f_ipc_r:
    i = i + 1
    line = line.strip()
    line_list = line.split(' ')
    #匹配不上返回 None
    match_str = re.search(pattern, line_list[0])
    if match_str is None:
        print('匹配不上：', line_list)
    else:
        if len(line_list) > 2:
            print('>2 error : ', line_list)
        elif len(line_list) == 2:
            sheet.write(i, 0, line_list[0]) #写入excel，i行0列
            sheet.write(i, 1, line_list[1]) #写入excel，i行1列
            sheet.write(i, 2, clc_list[i-1]) #写入excel，i行2列
        else:
            print('<2 error : ', line_list)

writebook.save('./data/input_manmaked.xlsx')#一定要记得保存

# 基于类目相似度的方法

In [1]:
import re
import os
import copy
import argparse
import warnings
import ipdb
import datetime

import pandas as pd
import numpy as np

#jieba用来分词和词性标注以及提取关键词
import jieba
import jieba.analyse
import jieba.posseg as pseg

#给结巴添加自定义词典
jieba.load_userdict("./word_embedding/vocab.txt")

#用来使用word2vec词向量进行匹配（近义词匹配）
import synonyms

#fuzzywuzzy库用来匹配字符串的相似度
#固定一个词不动；另一个词的长短对匹配无影响
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from tqdm import tqdm

warnings.filterwarnings("ignore")

import ipdb

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.585 seconds.
Prefix dict has been built successfully.


smart_open library not found; falling back to local-filesystem-only
[jieba] default dict file path ../data/vocab.txt


Building prefix dict from /home/hw/anaconda3/envs/transquest/lib/python3.8/site-packages/synonyms/data/vocab.txt ...
Loading model from cache /tmp/jieba.u0aee24de5e38baa78ebdb92867f204df.cache


>> Synonyms load wordseg dict [/home/hw/anaconda3/envs/transquest/lib/python3.8/site-packages/synonyms/data/vocab.txt] ... 


Loading model cost 1.215 seconds.
Prefix dict has been built successfully.


>> Synonyms on loading stopwords [/home/hw/anaconda3/envs/transquest/lib/python3.8/site-packages/synonyms/data/stopwords.txt] ...
[Synonyms] on loading vectors [/home/hw/anaconda3/envs/transquest/lib/python3.8/site-packages/synonyms/data/words.vector.gz] ...


In [2]:
#传入的参数/地址
input_dir = './data/input_manmaked.xlsx'
output_dir = './data/output_manmaked_373.xlsx'
nbest = 10  #就是top_k
auto_get_keywords = False  #与input_dir相关，如果input_dir输入的是已经分词的，则为False，否则为True
using_TF_IDF = False  #与auto_get_keywords相关，当auto_get_keywords为True时生效。表示是否使用TF_IDF提取关键词。

ipc_dir = './ipc/ipc_2019.txt'  #这个可以不用了
ipc_sentence_dir = './ipc/ipc_keywords_sentences.txt'  #只有句子级别的匹配才用得上(因为一对多，所以ipc会存在重复，这个没关系)

clc_dir = './clc/clc_keywords_sentences_dup.txt'  #clc的句子(重复的分类号需要处理一下)
clc_key_words_dir = './clc/clc_keywords_dup.txt'  #clc的关键词(重复的分类号需要处理一下)


#synonyms近义词匹配的个数，这是一个很重要的超参数，初步验证近义词个数太大效果变差（5->10 : 52%->45%）
#需要多尝试，默认为 10
nearby_number = 5


#四选一
only_using_fuzzy = False
only_using_synonyms_keywords, nearby_number = False, nearby_number
only_using_synonyms_sentence, no_symbol = False, False  #传入匹配的时候需不需要标点符号 ;
using_fuzzy_and_synonyms, nearby_number = True, nearby_number

In [3]:
#构造ipc字典 （现在已经可以不用了）
ipc_dict = {}
ipc_key_list = []

with open(ipc_dir, 'r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        line = line.split(' ')
        ipc_key_list.append(str(line[0]))
        ipc_dict[str(line[0])] = []
        for index in range(1, len(line)):
            ipc_dict[str(line[0])].append(str(line[index]))
            
#ipc_dict: {'A':['XXX'], 'B':['YYYY'], ...}


#构造ipc　sentence字典 （只有句子级别匹配需要用到）
ipc_sentence_dict = {}
ipc_sentence_key_list = []
ipc_sentence_value_list = []

with open(ipc_sentence_dir, 'r', encoding='utf-8') as f:
    key_i = 0
    for line in f:
        line = line.strip()
        line = line.split(' ')
        ipc_sentence_key_list.append(str(line[0]))
        ipc_sentence_value_list.append(str(line[1]))
        ipc_sentence_dict[str(key_i)] = []  #因为ipc的分类号有重复，所以不用ipc分类号作为key
        assert len(line) == 2, 'clc句子中存在空格！'
        
        ipc_sentence_dict[str(key_i)].append(str(line[1]))
        key_i = key_i + 1
        
#ipc_sentence_dict: {'0':['XXX'], '1':['YYYY'], ...}

In [4]:
#构造clc字典  clc的句子
clc_dict = dict()
clc_key_list = []
clc_value_list = []

with open(clc_dir, 'r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        line = line.split(' ')
        clc_key_list.append(str(line[0]))
        clc_value_list.append(str(line[1]))
        clc_dict[str(line[0])] = []
        assert len(line) == 2, 'clc句子中存在空格！'
        
        for index in range(1, len(line)):
            clc_dict[str(line[0])].append(str(line[index]))

#clc_dict: {'A':['XXX'], 'B':['YYYY'], ...}

In [5]:
#用来统计一个ipc包含多个clc的clc字典
match_clc_key_dict = dict()

for ipc_key_i, target in enumerate(ipc_sentence_key_list):
    match_clc_key_dict[str(ipc_key_i)] = []
    for ipc_key_j, every_ipc_key in enumerate(ipc_sentence_key_list):
        if every_ipc_key == target:
            #key是数字序号，和clc分类号顺序是一致的 ，有1000条数据
            match_clc_key_dict[str(ipc_key_i)].append(clc_key_list[ipc_key_j])

#用于计算IPC每一类别包含的数量
ipc_section_number_dict = {
    'A' : 0,
    'B' : 0,
    'C' : 0,
    'D' : 0,
    'E' : 0,
    'F' : 0,
    'G' : 0,
    'H' : 0
}
for ipc_key_i in ipc_sentence_key_list:
    ipc_section_number_dict[str(ipc_key_i[0])] += 1

In [6]:
#定义一个clc字典，用来存储匹配得分，因为clc对重复的数据进行了+1处理，所以是1000条数据
rank_clc_dict = dict()
for key, value in clc_dict.items():
    rank_clc_dict[str(key)] = 0
    
#clc_dict: {'A':0, 'B':0, ...}

In [7]:
#构造clc key words字典, 因为clc对重复的数据进行了+1处理，所以是1000条数据
clc_key_words_dict = dict()
clc_key_words_key_list = []
save_null_list_dict = {}

with open(clc_key_words_dir, 'r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        line = line.split(' ')
        clc_key_words_key_list.append(str(line[0]))
        clc_key_words_dict[str(line[0])] = []
        save_null_list_dict[str(line[0])] = []
        assert len(line) == 2
        
        key_words_value_list = line[1].split(';')
        for index in range(len(key_words_value_list)):
            if str(key_words_value_list[index]) != '':
                clc_key_words_dict[str(line[0])].append(str(key_words_value_list[index]))
            else:
                save_null_list_dict[str(line[0])].append(str(key_words_value_list[index]))
                
print(len(save_null_list_dict))
assert len(clc_dict) == len(clc_key_words_dict), 'clc句子和关键词的字典大小需要一致！'

#clc_key_words_dict: {'A':['XXX', 'WW'], 'B':['YYYY', 'QQ', 'E'], ...}

373


In [8]:
#用来统计字典的值有没有超过1的
for k,v in clc_dict.items():
    if len(v) > 1:
        print(k, v)
        
for k,v in ipc_dict.items():
    if len(v) > 1:
        print(k, v)

In [9]:
#用来统计最长的ipc和clc的分类号
max_len = 0
max_len_value = ''
for io in ipc_key_list:
    if len(io) > max_len:
        max_len = len(io)
        max_len_value = io
max_len, max_len_value

(12, 'F21W131/4035')

In [10]:
#匹配ipc分类号的正则表达式
pattern = '[A-H]{1}[0-9]{2}[A-Z]{1}[0-9]{1,4}\/[0-9]{2,6}'
#读入要匹配的输入文件，路径注意转义 
df = pd.read_excel(input_dir, sheet_name=None)
#i用来标识当前是第几个子表(sheet)
i = 0
#向同一个Excel中写入多个子表(sheet)
writer = pd.ExcelWriter(output_dir)


#df.keys()为每个子表sheet的名称，df.values()为每个子表sheet的内容
for sheet in df.values():
    #在子表中新增三列，并赋初值
    col_name = sheet.columns.tolist()
    col_name.insert(len(col_name)+1, 'CLC分类号汇总')
    sheet = sheet.reindex(columns = col_name)
    sheet['CLC分类号汇总'] = ['0' for f_z in sheet['IPC分类号']]
    
    col_name.insert(len(col_name)+2, 'CLC描述')
    sheet = sheet.reindex(columns = col_name)
    sheet['CLC描述'] = [' ' for f_z in sheet['CLC描述']]
    
    col_name.insert(len(col_name)+3, 'CLC匹配频次')
    sheet = sheet.reindex(columns = col_name)
    sheet['CLC匹配频次'] = ['0' for f_z in sheet['CLC匹配频次']]
    
    #读出 'IPC分类号'
    sheet1 = sheet['IPC分类号']
    input_ipc = []  #存储要匹配的ipc分类号
    is_matched_flag = []  #标记ipc分类号在ipc字典中是否可以查到
    
    #存储匹配正确的数量(只取匹配度最高的)
    correct = 0
    
    error_match_ipc_clc_pair = []

    #存储TopK中匹配正确的数量
    top_k_correct = 0
    
    #存储top2-TopK中匹配正确的数量(包含了topK)
    top2_to_topk_correct_dict = {
        '2' : 0,
        '3' : 0,
        '4' : 0,
        '5' : 0,
        '6' : 0,
        '7' : 0,
        '8' : 0,
        '9' : 0,
        '10' : 0
    }
    
    #存储每一个IPC类别的正确个数
    ipc_section_correct_dict = {
        'A' : 0,
        'B' : 0,
        'C' : 0,
        'D' : 0,
        'E' : 0,
        'F' : 0,
        'G' : 0,
        'H' : 0
    }
    
    topk_error_match_ipc_clc_pair = []
    
    #遍历当前子表(sheet)的每一个ipc分类号
    for j in range(len(sheet1)):
        #取出IPC分类号，并转成字符串
        sheet1[j] = str(sheet1[j])
        line = sheet1[j].strip()
        
        #返回line中第一个匹配上的ipc号，没有则返回None
        match_str = re.search(pattern, line)
        
        #匹配上了，并且存在ipc字典中（全部取excel表中的数据，if判断直接为False）
        if False and (match_str is not None) and (str(match_str.group()) in ipc_key_list):
            is_matched_flag.append(True)
            input_ipc.append(str(match_str.group()))
        else:
            is_matched_flag.append(False)
            #print('当前第{}个子表，第{}个IPC号：[{}]格式不正确或者ipc字典中没有这个ipc分类号！将自动提取sheet的“IPC描述”内容作为当前的匹配字符串！'.format(i+1, j+1, line))
            #print()
            input_ipc.append(str(sheet['IPC描述'][j]))
            
    '------------句子级别匹配---------------'
    if only_using_synonyms_sentence:
        input_ipc = []
        for ipc_kk, ipc_vv_list in ipc_sentence_dict.items():
            input_ipc.append(ipc_vv_list)
    
    '------------句子级别匹配---------------'
    
            
    for ipc_i, current_input_ipc in enumerate(input_ipc):
        #每个ipc分类号都要重新初始化clc字典，用来存储匹配得分
        for key, value in rank_clc_dict.items():
            rank_clc_dict[str(key)] = 0
            
        '''---------------这一部分已经不用了------------------'''
        #匹配上了，则到ipc字典中取（is_matched_flag中全为False）
        if is_matched_flag[ipc_i]:
            current_ipc_description = []  #存储当前ipc的描述
            
            section_ipc_description = []  #存储当前ipc“部”的描述
            main_class_ipc_description = []#存储当前ipc“大类”的描述
            sub_class_ipc_description = []#存储当前ipc“小类”的描述
            main_group_ipc_description = []#存储当前ipc“主组”的描述
            #存储经过处理后的ipc描述（包含上述所有的描述）
            ipc_description = []

            #从ipc字典中获得当前输入的ipc描述，之后将当前ipc的所有父节点（上一级）的描述并入当前ipc的描述中
            current_ipc_description = ipc_dict.get(current_input_ipc)

            #1 取“部”的描述
            section_ipc_description = ipc_dict.get(current_input_ipc[0])
            ipc_description.extend(section_ipc_description)

            #2 取“大类”的描述
            main_class_ipc_description = ipc_dict.get(current_input_ipc[0:3])
            ipc_description.extend(main_class_ipc_description)

            #3 取“小类”的描述
            sub_class_ipc_description = ipc_dict.get(current_input_ipc[0:4])
            ipc_description.extend(sub_class_ipc_description)

            #4 判断当前 current_input_ipc 是否为 00 结尾
            #为 00 结尾：直接结束(得到分组的描述)
            #不为 00 结尾：将 00 结尾的描述并入当前ipc的描述中
            two_item_list = current_input_ipc.split('/')
            if str(two_item_list[1]) == '00':
                ipc_description.extend(current_ipc_description)
            else:
                #4 取“主组”的描述
                main_group_ipc_description = ipc_dict.get(str(two_item_list[0]) + '/00')
                ipc_description.extend(main_group_ipc_description)
                ipc_description.extend(current_ipc_description)

            #将ipc_description列表中的描述全部连接起来，作为一个ipc匹配（查询）字符串
            ipc_query_string = ''
            for each_ipc_description in ipc_description:
                ipc_query_string = ipc_query_string + str(each_ipc_description)
        
            '''---------------这一部分已经不用了------------------'''
        #否则没有匹配上。
        else:
            #如果没有匹配上，则直接取sheet表中的“IPC描述”作为当前ipc匹配（查询）字符串
            ipc_query_string = current_input_ipc
        
        if auto_get_keywords:
            if using_TF_IDF:
                #使用TF-IDF算法和TextRank算法提取关键词，并取两者的并集，可以指定提取关键词的个数(topK)和提取关键词的词性(allowPOS)
                list1 = jieba.analyse.extract_tags(ipc_query_string, topK=30, withWeight=False, \
                                                    allowPOS=allow_pos_characters)
                list2 = jieba.analyse.textrank(ipc_query_string, topK=30, withWeight=False, \
                                                allowPOS=allow_pos_characters)

                #取并集并去重
                dedup_ipc_key_words_list = list(set(list1).union(set(list2)))
            
            #直接使用jieba分词作为关键字
            else:
                ipc_query_string_list = jieba.lcut(ipc_query_string, HMM=True)
                ipc_query_string_list = list(set(ipc_query_string_list))
                vv_list = []
                for vv_str1 in ipc_query_string_list:
                    if vv_str1 != ';' and vv_str1 != '（' and vv_str1 != '）' and vv_str1 != '，' \
                        and vv_str1 != '、' and vv_str1 != '，':
                        vv_list.append(vv_str1)

                dedup_ipc_key_words_list = vv_list

        else:
            if only_using_synonyms_sentence:
                dedup_ipc_key_words_list = ipc_query_string
            else:
                dedup_ipc_key_words_list = list(set(ipc_query_string.rstrip(';').split(';')))
        
        ''' print('第', ipc_i, '个ipc关键词个数为：', len(dedup_ipc_key_words_list)) '''
        ''' print('第', ipc_i, '个ipc关键词为：', dedup_ipc_key_words_list) '''
        
        if only_using_fuzzy:
            #对ipc中每一个关键字在clc字典中作查询/匹配
            for each_ipc_key_words in dedup_ipc_key_words_list:
                for clc_key, clc_value in clc_dict.items():
                    #字形匹配上了，该clc分类号+1
                    if each_ipc_key_words in clc_value[0]:
                        rank_clc_dict[clc_key] = rank_clc_dict[clc_key] + 1
                    #匹配不上，继续
                    else:
                        continue
                        
                        
        if only_using_synonyms_keywords:
            #对ipc的句子与clc句子进行匹配，得分最高的即为相互映射
            for each_ipc_key_words in dedup_ipc_key_words_list:
                for clc_key, clc_value in clc_key_words_dict.items():
                    for clc_value0 in clc_value:
                        #获取ipc的近义词列表
                        ipc_nearby_words_tuple = synonyms.nearby(each_ipc_key_words, nearby_number)
                        ipc_nearby_words_list = ipc_nearby_words_tuple[0]  #元组第一个为近义词，第二个为近义词距离得分
                        #当OOV时，返回[]，则忽略匹配
                        if len(ipc_nearby_words_list) > 0:
                            #近义词匹配上了，该clc分类号+1
                            if clc_value0 in ipc_nearby_words_list:
                                rank_clc_dict[clc_key] = rank_clc_dict[clc_key] + 1
                                break
                            #匹配不上，继续
                            else:
                                continue
 
        
        if only_using_synonyms_sentence:
            #对ipc中每一个关键字在clc字典中作查询/匹配
            for clc_key, clc_sentence_list in clc_dict.items():
                if no_symbol:#去掉标点符号　;
                    s1 = ''.join(''.join(dedup_ipc_key_words_list).rstrip(';').split(';'))
                    s2 = ''.join(''.join(clc_sentence_list).rstrip(';').split(';'))
                else:
                    s1 = ''.join(dedup_ipc_key_words_list)
                    s2 = ''.join(clc_sentence_list)
                #利用synonyms进行word2vec句子语义匹配
                s_r = synonyms.compare(s1, s2, seg=True, \
                                       ignore=True, \
                                       stopwords=False
                                      )
                rank_clc_dict[clc_key] = s_r
                            
            
        #1月6日进行了修改，准确率有提升    
        if using_fuzzy_and_synonyms:
            #对ipc中每一个关键字在clc字典中作查询/匹配
            for each_ipc_key_words in dedup_ipc_key_words_list:
                for clc_key, clc_value in clc_dict.items():
                    #字形匹配上了，该clc分类号+1
                    if each_ipc_key_words in clc_value[0]:
                        rank_clc_dict[clc_key] = rank_clc_dict[clc_key] + 1
                    #匹配不上，继续用近义词匹配
                    else:
                        clc_value_description = clc_key_words_dict.get(clc_key)
                        #获取ipc关键词的近义词
                        ipc_nearby_words_tuple = synonyms.nearby(each_ipc_key_words, nearby_number)
                        #近义词匹配上了，该clc分类号+1
                        ipc_nearby_words_list = ipc_nearby_words_tuple[0]  #元组第一个为近义词，第二个为近义词距离得分
                        
                        #如果当前ipc关键词在synonyms中找不到关键词，即属于oov，synonyms返回 ([], [])；
                        #则当前ipc关键词不进行匹配，这里出现oov不会报警告。
                        if len(ipc_nearby_words_list) > 0:
                            for ipc_value0 in ipc_nearby_words_list:
                                if ipc_value0 in clc_value_description:
                                    rank_clc_dict[clc_key] = rank_clc_dict[clc_key] + 1
                                    break
                                #匹配不上，继续
                                else:
                                    continue
            
            #还属于 using_fuzzy_and_synonyms  --方案一 （这个效果最差）
            #for clc_k_1, clc_v_1 in clc_key_words_dict.items():
            #    rank_clc_dict[clc_k_1] = rank_clc_dict[clc_k_1]/len(clc_v_1)
                
            #还属于 using_fuzzy_and_synonyms  --方案二 （这个结合方案三效果最好）
            ipc_to_match_sentence = ipc_sentence_dict[str(ipc_i)][0]
            for clc_k_1, clc_v_1 in clc_dict.items():
                rank_clc_dict[clc_k_1] = rank_clc_dict[clc_k_1] + (fuzz.ratio(ipc_to_match_sentence, clc_v_1[0])/100)
        
        #方案三（仅使用这个效果也还不错，加上句子级别的匹配效果更差）
        #ipc_to_match_sentence = ipc_sentence_dict[str(ipc_i)][0]
        #for clc_k_1, clc_v_1 in clc_dict.items():
        #    rank_clc_dict[clc_k_1] = fuzz.ratio(ipc_to_match_sentence, clc_v_1[0])
            #rank_clc_dict[clc_k_1] = (fuzz.ratio(ipc_to_match_sentence, clc_v_1[0])/100) + synonyms.compare(ipc_to_match_sentence, \
            #                            clc_v_1[0], seg=True, ignore=True, stopwords=False)
        
            
        '-----------------------------不需要后处理了------------------------------'            
        #上述执行完后，rank_clc_dict就存有当前ipc匹配后的得分，
        #这时需要进行rank_clc_dict的后处理，得到最终排序前几的结果作为最终结果
        #for rank_clc_key1, rank_clc_value1 in rank_clc_dict.items():
            #不等于0才需要赋值
        #    if rank_clc_value1 != 0:
        #        for rank_clc_key2, rank_clc_value2 in rank_clc_dict.items():
        #            if (rank_clc_key1 != rank_clc_key2) and (rank_clc_key1[0] == rank_clc_key2[0]) and \
        #                (rank_clc_key1 in rank_clc_key2) and (len(rank_clc_key1) <= len(rank_clc_key2)):
        #                rank_clc_dict[rank_clc_key2] = rank_clc_dict[rank_clc_key2] + rank_clc_dict[rank_clc_key1]
        '-----------------------------不需要后处理了------------------------------' 
        
        
        #对rank_clc_dict按照值大小进行排序，输出前nbest个结果
        #rank_clc_dict_result_list，结果由大到小，包含每一条CLC分类号匹配的结果
        rank_clc_dict_result_list = sorted(rank_clc_dict.items(), \
                                            key=lambda item:item[1], \
                                            reverse=True
                                            )
        

        
        nbest_clc_key = ''
        nbest_clc_value = ''
        nbest_clc_frequency = ''
        for nbest_i, value in enumerate(rank_clc_dict_result_list):
            #只取前nbest个结果
            if nbest_i == nbest:
                break
            else:   
                nbest_clc_key = nbest_clc_key + str(value[0]) + ';'
                nbest_clc_value0 = ''
                for value0 in clc_dict.get(str(value[0])):
                    nbest_clc_value0 = nbest_clc_value0 + value0
                nbest_clc_value = nbest_clc_value + nbest_clc_value0 + ';'
                nbest_clc_frequency = nbest_clc_frequency + str(value[1]) + ';'

        clc_key_n_best_list = nbest_clc_key.rstrip(';').split(';')
        nbest_clc_frequency_list = nbest_clc_frequency.rstrip(';').split(';')
        #找出前几个匹配得分最高的，相同的出来
        clc_key_n_best_list_first = []
        clc_key_n_best_list_first.append(clc_key_n_best_list[0])
        for clc_key_n_best_i in range(1, len(clc_key_n_best_list)):
            if nbest_clc_frequency_list[clc_key_n_best_i] == nbest_clc_frequency_list[0]:
                clc_key_n_best_list_first.append(clc_key_n_best_list[clc_key_n_best_i])
            else:
                break

        #匹配：只要两个列表有交集就是匹配上了
        #for clc_key_n_best_list_first_1 in clc_key_n_best_list_first:   #--这个方案可以使准确率上升很多，但是不能用
        if clc_key_n_best_list[0] in match_clc_key_dict[str(ipc_i)]:  #--还是这个方案最稳妥
        #if clc_key_n_best_list_first_1 in match_clc_key_dict[str(ipc_i)]:
            correct = correct + 1
            ipc_section_correct_dict[str(ipc_sentence_key_list[ipc_i][0])] += 1
            #break
        else:
            error_match_ipc_clc_pair.append(str(sheet['IPC分类号'][ipc_i]) + str(clc_key_n_best_list[0]))

        #匹配：只要两个列表有交集就是匹配上了    
        for each_clc_key_1 in match_clc_key_dict[str(ipc_i)]:
            if str(each_clc_key_1) in clc_key_n_best_list:
                top_k_correct = top_k_correct + 1
                break
            else:
                topk_error_match_ipc_clc_pair.append(str(sheet['IPC分类号'][ipc_i]) + str(clc_key_n_best_list[0]))

        #计算tok2-topK每一个的准确率
        for topk_index in range(1, nbest):
            for each_clc_key_1 in match_clc_key_dict[str(ipc_i)]:
                if str(each_clc_key_1) in clc_key_n_best_list[:topk_index+1]:
                    top2_to_topk_correct_dict[str(topk_index+1)] += 1
                    break

        sheet.loc[ipc_i, 'CLC分类号汇总'] = nbest_clc_key
        sheet.loc[ipc_i, 'CLC描述'] = nbest_clc_value
        sheet.loc[ipc_i, 'CLC匹配频次'] = nbest_clc_frequency
        
        #每隔 10 个ipc分类号打印一次时间
        curr_time = datetime.datetime.now()
        time_str = datetime.datetime.strftime(curr_time,'%Y-%m-%d %H:%M:%S')
        if ipc_i % 50 == 0:
            print('当前时间【{}】已经执行到第 {} 个ipc分类号了！'.format(time_str, ipc_i+1))
            print()

        
    #将子sheet写入输出文件中
    sheet.to_excel(writer, sheet_name=list(df)[i], index=False)
    i = i + 1

    
#所有子sheet写完后保存并关闭                 
writer.save()
writer.close()

#计算准确率，TopK准确率
ipc_count_number = len(input_ipc)

acc = correct/ipc_count_number
top_k_acc = top_k_correct/ipc_count_number
print('\nFinal acc: {:.2f}% .'.format(acc*100))
print('\nFinal top_{}_acc: {:.2f}% .'.format(nbest, top_k_acc*100))


#结果统计
print('\n总数：{}\ntop1 正确个数：{}\ntop1 错误个数：{}\n'.format(ipc_count_number, correct, \
                                                       ipc_count_number-correct))

print('总数：{}\ntopk 正确个数：{}\ntopk 错误个数：{}\n'.format(ipc_count_number, top_k_correct, \
                                                       ipc_count_number-top_k_correct))

print('\n--------------------\n')
for topk_name, top_correct_num in top2_to_topk_correct_dict.items():
    print('\nFinal top_{}_acc: {:.2f}% .'.format(topk_name, (top_correct_num/ipc_count_number)*100))

print('\n--------------------\n')
for topk_name, top_correct_num in top2_to_topk_correct_dict.items():
    print('总数：{}\ntop{} 正确个数：{}\ntop{} 错误个数：{}\n'.format(ipc_count_number, topk_name, top_correct_num, topk_name, ipc_count_number-top_correct_num))

print('\n--------------------\n')
#计算每一个IPC类别的正确个数和正确率（只有top1的）
for sec_i, sec_cor in ipc_section_correct_dict.items():
    print('类别：{}，总数为：{}，正确个数为：{}，错误个数为：{}，正确率为：{:.2f}%\n'.format(sec_i, ipc_section_number_dict[sec_i], sec_cor, ipc_section_number_dict[sec_i]-sec_cor, (sec_cor/ipc_section_number_dict[sec_i])*100))

print()
print('程序执行完毕！')

当前时间【2021-02-25 23:14:34】已经执行到第 1 个ipc分类号了！

当前时间【2021-02-25 23:15:14】已经执行到第 51 个ipc分类号了！

当前时间【2021-02-25 23:15:45】已经执行到第 101 个ipc分类号了！

当前时间【2021-02-25 23:16:13】已经执行到第 151 个ipc分类号了！

当前时间【2021-02-25 23:16:31】已经执行到第 201 个ipc分类号了！

当前时间【2021-02-25 23:16:50】已经执行到第 251 个ipc分类号了！

当前时间【2021-02-25 23:17:13】已经执行到第 301 个ipc分类号了！

当前时间【2021-02-25 23:17:36】已经执行到第 351 个ipc分类号了！


Final acc: 64.08% .

Final top_10_acc: 94.91% .

总数：373
top1 正确个数：239
top1 错误个数：134

总数：373
topk 正确个数：354
topk 错误个数：19


--------------------


Final top_2_acc: 78.82% .

Final top_3_acc: 84.99% .

Final top_4_acc: 87.94% .

Final top_5_acc: 90.08% .

Final top_6_acc: 91.15% .

Final top_7_acc: 92.23% .

Final top_8_acc: 93.57% .

Final top_9_acc: 94.64% .

Final top_10_acc: 94.91% .

--------------------

总数：373
top2 正确个数：294
top2 错误个数：79

总数：373
top3 正确个数：317
top3 错误个数：56

总数：373
top4 正确个数：328
top4 错误个数：45

总数：373
top5 正确个数：336
top5 错误个数：37

总数：373
top6 正确个数：340
top6 错误个数：33

总数：373
top7 正确个数：344
top7 错误个数：29

总数：373